In [66]:
import pandas as pd
import numpy as np
import random as rd
import math

In [67]:
df = pd.read_excel('Data_Tugas_Akhir.xlsx', sheetname='Data_latih')
target = pd.read_excel('Data_Tugas_Akhir.xlsx', sheetname='Target')

In [68]:
dfNormalisasi = (df - df.mean())/(df.max() - df.min())
ValueTarget = np.array(target['target'])

In [69]:
alfa = 0.1
input_unit = 6
output_unit = 3
hidden_layer = 2

In [70]:
target_data_latih = [
    [1, 0, 0], #Kelas1
    [0, 1, 0], #Kelas2
    [0, 0, 1]  #Kelas3
]

In [71]:
#InputanAwalBebasUntukProsesNguyen
VijLama = np.random.uniform(-0.5, 0.5, (hidden_layer, input_unit))

In [72]:
#SetNilaiBeta
valueB = 0.7*((hidden_layer)**1/input_unit)

In [73]:
def valueMutlak(hidden_layer, dataInput):
    MatriksKosong = []
    for col in range (0, hidden_layer):
        for value in dataInput:
            valueV = math.sqrt(value[col]**2)
        MatriksKosong.append(valueV)
    return MatriksKosong

In [74]:
MutlakV = valueMutlak(hidden_layer, VijLama)

In [75]:
#Method Untuk Mendapatkan Weigth Berdasarkan Nguyen
def NguyenWidrow(mutlak, valueB, dataInput):
    bantu = []
    ValueNewV = []
    for baris in range(0, hidden_layer):
        for kolom in range(0, input_unit):
            valueV = valueB * dataInput[baris][kolom] / MutlakV[baris]
            bantu.append(valueV)
    ValueNewV.append(bantu)
    ValueBaru = np.reshape(ValueNewV, (2,6))
    return ValueBaru

In [76]:
#Methode untuk mendapatkan Bias Hidden Layer
def getValueBiasNew(valueB):
    BiasBaru = np.random.uniform(-valueB, valueB, (2))
    return BiasBaru

FUNGSI AKTIVASI

In [77]:
def arctangenFunction(x):
    phi = 3.14159265358979323846
    ValueTan = math.atan(x)
    return (2 /phi) * ValueTan
    

In [78]:
def arctangenFunctionDerivative(x):
    phi = 3.14159265358979323846
    Value = 1 / 1 + math.pow(x, 2)
    return (2 /phi) * Value

In [79]:
def fungsi_aktivasi(x):
#     nilai = x
    Value = 1+ np.exp(-x)
    return 1/Value

In [80]:
def fungsi_aktivasi_turunan(x) :
    pengkali = 1 - fungsi_aktivasi(x)
    return fungsi_aktivasi(x)*pengkali

INISIALISASI BOBOT

In [81]:
#Bias Hidden Layer
ValueV0jHidden = getValueBiasNew(valueB)

In [82]:
#Bobot Hidden Layer
ValueVijHidden = NguyenWidrow(MutlakV, valueB, VijLama) 

In [83]:
#BobotOutputLayer
ValueWjkOutput = np.random.uniform(-0.5, 0.5, (output_unit, hidden_layer))

In [84]:
#Bias Output Layer
ValueW0kOutput = np.random.uniform(-0.5, 0.5, (3))

PROSES

In [85]:
def getFeedForward(dataLatih, Bobot_Hidden, Bias_Hidden, Bias_Output, Bobot_Output):
    ValueZ = []
    ValueY = []
    ValueZin = []
    ValueYin = []
    Bantu = []
    #Mencari Nilai Zin = Vij * x + V0j
    #Sekaligus Mendapatkan Nilai Z yang telah dilakukan Aktivasi
    for key, value in dataLatih.iterrows():
        for baris in range(0, hidden_layer):
            for kolom in range(0, input_unit):
                NilaiZin = sum(Bobot_Hidden[baris][kolom] * value)
            bantu = NilaiZin + Bias_Hidden[baris]
            Bantu.append(bantu)
            z = [fungsi_aktivasi(i) for i in Bantu]
        ValueZin.append(Bantu)
        Bantu = []
        ValueZ.append(z)

    #Mencari nilai Yin = Wjk * z + W0k
    #Sekaligus Mendapatkan Nilai Y yang telah dilakukan Aktivasi
    NilaiBantu = 0
    for key, value in dataLatih.iterrows():
        for baris in range (len(Bobot_Output)):
            for kolom in range (len(Bobot_Output[0])):
                NilaiYin = ValueZ[baris][kolom] * Bobot_Output[baris][kolom]
                NilaiBantu += NilaiYin
            NilaiYin = NilaiBantu + Bias_Output[baris]
            Bantu.append(NilaiYin)
            y = [fungsi_aktivasi(i) for i in Bantu]
        ValueYin.append(Bantu)
        Bantu = []
        ValueY.append(y)
        
#     print(Val)
#     print(ValueY)
    return ValueZ, ValueY, ValueZin, ValueYin

In [86]:
ValueZ, ValueY, ValueZin, ValueYin = getFeedForward(dfNormalisasi, ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput)

In [87]:
def BackPreprogration(dfNormalisasi,Yin, ValueY, target, alfa, ValueZ, ValueWjk, Zin):
    ValueDk = []
    ValueDw = []
    ValueDw0 = []
    ValueDin = []
    ValueDz = []
    ValueDv = []
    ValueDv0 = []
    ValueYinAkitvasiTurunan = []
    ValueZinAkitvasiTurunan = []
    Bantu = []
    Bantu2 = []
    
    TargetComvert = [
    [1, 0, 0], #Kelas1
    [0, 1, 0], #Kelas2
    [0, 0, 1]  #Kelas3
]
    for baris in range (len(Yin)):
        YinNew = [fungsi_aktivasi_turunan(i) for i in Yin[baris]]
        ValueYinAkitvasiTurunan.append(YinNew)
        
#     Hitung  δk
    for baris in range (len(ValueY)):
        for kolom in range(len(ValueY[0])):
            if(target[baris] == 1):
                NilaiDk = (TargetComvert[0][kolom] - ValueY[baris][kolom]) * ValueYinAkitvasiTurunan[baris][kolom]
            elif(target[baris] == 2):
                NilaiDk = (TargetComvert[1][kolom] - ValueY[baris][kolom]) * ValueYinAkitvasiTurunan[baris][kolom]
            elif(target[baris] == 3):
                NilaiDk = (TargetComvert[2][kolom] - ValueY[baris][kolom]) * ValueYinAkitvasiTurunan[baris][kolom]
            Bantu.append(NilaiDk)
        ValueDk.append(Bantu)
        Bantu = []
        
    #Hitung ΔWjk
    for baris in range (len(ValueDk)):
        for kolom in range (len(ValueDk[0])):
            for kolom1 in range(len(ValueZ[0])):
                NilaiDw = ValueDk[baris][kolom] * ValueZ[baris][kolom1] * alfa
                Bantu.append(NilaiDw)
            Bantu2.append(Bantu)
            Bantu = []
        ValueDw.append(Bantu2)
        Bantu2 = []
    
    #Hitung ΔW0k alfa * Dk
    for baris in range(len(ValueDk)):
        NilaiDw0 = [i * alfa for i in ValueDk[baris]]
        ValueDw0.append(NilaiDw0)
#     print(ValueDw0)
    
    transpose_wjk = np.transpose(ValueWjk)
    
#     Hitung δin = δk * Wjk
    for i in range (len(dfNormalisasi)):
        NilaiBantu = 0
        for baris in range (len(transpose_wjk)):
            for kolom in range (len(transpose_wjk[0])):
                NilaiDin = ValueDk[i][kolom] * transpose_wjk[baris][kolom]
                NilaiBantu += NilaiDin
            Bantu.append(NilaiBantu)
        ValueDin.append(Bantu)
        Bantu = []
        NilaiBantu = 0
#     print(ValueDin)
    #Hitung Dz
    for baris in range(len(Zin)):
        for kolom in range (len(Zin[0])):
            ZinAktivasiTurunan = fungsi_aktivasi_turunan(Zin[baris][kolom])
            Bantu.append(ZinAktivasiTurunan)
        ValueZinAkitvasiTurunan.append(Bantu)
        Bantu = []
#     print(ValueZinAkitvasiTurunan)
    for baris in range(len(ValueDin)):
        for kolom in range(len(ValueDin[0])):
            NilaiDz = ValueDin[baris][kolom] * ValueZinAkitvasiTurunan[baris][kolom]
            Bantu.append(NilaiDz)
        ValueDz.append(Bantu)
        Bantu = []
        
    #Hitung ΔVij
    Data = np.array(dfNormalisasi)
    for baris in range (len(Data)):
        for kolom in range(len(ValueDz[0])):
            for kolom1 in range(0, input_unit):
                NilaiVij = alfa * Data[baris][kolom1] * ValueDz[baris][kolom]
                Bantu.append(NilaiVij)
            Bantu2.append(Bantu)
            Bantu = []
        ValueDv.append(Bantu2)
        Bantu2 = []
        
    #Hitung ΔV0j
    for baris in range (len(ValueDz)):
        for kolom in range(len(ValueDz[0])):
            NilaiV0j = alfa * ValueDz[baris][kolom]
            Bantu.append(NilaiV0j)
        ValueDv0.append(Bantu)
        Bantu = []
        
    return ValueDw, ValueDw0, ValueDv, ValueDv0

In [88]:
ValueDw, ValueDw0, ValueDv, ValueDv0 = BackPreprogration(dfNormalisasi,ValueYin, ValueY, ValueTarget, alfa, ValueZ, ValueWjkOutput, ValueZin)

In [89]:
def UpdateBobot(ValueWjk, ValueDw, ValueDw0, ValueW0, ValueDv0, ValueV0, ValueVij, ValueDv):
    Miu = 0.175
    
    #UpdateBobotVij
    for baris in range (len(ValueDv)):
        for kolom1 in range (len(ValueDv[0])):
            for kolom in range(len(ValueVij[0])):
                NilaiVbaru = ValueDv[baris][kolom1][kolom] + (Miu * ValueVij[kolom1][kolom])
                ValueVij[kolom1][kolom] = NilaiVbaru
#     print(ValueVij)
    #UpdateBobotW0j
    for baris in range(len(ValueDv0)):
        for kolom in range(len(ValueDv0[0])):    
            NilaiW0Baru = (ValueDv0[baris][kolom] + ValueV0[kolom])
            ValueV0[kolom] = NilaiW0Baru
            
    #UpdateBiasNewW0
    for baris in range(len(ValueDw0)):
        for kolom in range(len(ValueDw0[0])):
            NilaiBiasHiddenBaru = ValueDw0[baris][kolom] + ValueW0[kolom]
            ValueW0[kolom] = NilaiBiasHiddenBaru
#     print(ValueW0)

    #UpdateBiasNewWjk
    for baris in range(len(ValueDw)):
        for indeks2d in range(len(ValueDw[0])):
            for kolom in range(len(ValueWjk[0])):
                NilaiWBaru = ValueDw[baris][indeks2d][kolom] + (Miu*ValueWjk[indeks2d][kolom])
                ValueWjk[indeks2d][kolom] = NilaiWBaru
#     print(ValueWjk)
    return ValueVij,ValueV0, ValueW0, ValueWjk

In [90]:
def CheckError(ValueY, target):
    TargetComvert = [
        [1, 0, 0], #Kelas1
        [0, 1, 0], #Kelas2
        [0, 0, 1]  #Kelas3
    ]
    Error = []
    Bantu = []
    for baris in range (len(ValueY)):
        for kolom in range(len(ValueY[0])):
            if(target[baris] == 1):
                NilaiError = math.pow((ValueY[baris][kolom] - TargetComvert[0][kolom]),2)
            elif(target[baris] == 2):
                NilaiError = math.pow((ValueY[baris][kolom] - TargetComvert[1][kolom]),2)
            elif(target[baris] == 3):
                NilaiError = math.pow((ValueY[baris][kolom] - TargetComvert[2][kolom]),2)
            Bantu.append(NilaiError)
        Error.append(Bantu)
        Bantu = []
    Jumlah_ErrorPerBaris = 0
    JumlahKeseluruhan = []
    for baris in range(len(Error)):
        for kolom in range(len(Error[0])):
            Jumlah_ErrorPerBaris += Error[baris][kolom]
        JumlahKeseluruhan.append(Jumlah_ErrorPerBaris)
    TotalError1Epoch = 0
    for baris in range(len(JumlahKeseluruhan)):
        TotalError1Epoch += JumlahKeseluruhan[baris]
    NilaiRMSE = math.sqrt(TotalError1Epoch / len(JumlahKeseluruhan))
    return NilaiRMSE
#     print(NilaiRMSE)
        

In [91]:
def MencariBobotOptimal(dfNormalisasi, ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput):
    Epoch = 0
    MaxEpoch = 1000
    updateError = 0
    NilaiRMSE = 0
    while(Epoch < MaxEpoch or updateError < NilaiRMSE):
        ValueZ, ValueY, ValueZin, ValueYin = getFeedForward(dfNormalisasi, ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput)
        ValueDw, ValueDw0, ValueDv, ValueDv0 = BackPreprogration(dfNormalisasi,ValueYin, ValueY, ValueTarget, alfa, ValueZ, ValueWjkOutput, ValueZin)
        ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput = UpdateBobot(ValueWjkOutput, ValueDw, ValueDw0, ValueW0kOutput, ValueDv0, ValueV0jHidden, ValueVijHidden, ValueDv)
        NilaiRMSE = CheckError(ValueY, ValueTarget)
        updateError = NilaiRMSE
        Epoch += 1
    print(Epoch)
    
    return ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput

In [92]:
NilaiVij, NilaiV0, NilaiW0, NilaiWjk = MencariBobotOptimal(dfNormalisasi, ValueVijHidden, ValueV0jHidden, ValueW0kOutput, ValueWjkOutput)

1000


In [97]:
DataUji = pd.read_excel('Data_Tugas_Akhir.xlsx', sheetname='Data_Uji')
DataUjiNormalisasi = (DataUji - DataUji.mean())/(DataUji.max() - DataUji.min())

In [98]:
ValueZDataUji, ValueYUji, ValueZinUji, ValueYinDataUji = getFeedForward(DataUjiNormalisasi, NilaiVij, NilaiV0, NilaiW0, NilaiWjk)

In [99]:
def Pengujian(dataLatih,ValueVij, ValueV0, ValueW0, ValueWjk, ValueZin, ValueZ):
    Bantu = []
    ValueZin = []
    ValueYin = []
    ValueZ = []
    ValueY = []
    
    #Perhitungan Input Pada Hidden Neuron
    #Zin = Vij * x + V0j
    for key, value in dataLatih.iterrows():
        for baris in range(0, hidden_layer):
            for kolom in range(0, input_unit):
                NilaiZin = sum(ValueVij[baris][kolom] * value)
            bantu = NilaiZin + ValueV0[baris]
            Bantu.append(bantu)
            z = [fungsi_aktivasi(i) for i in Bantu]
        ValueZin.append(Bantu)
        Bantu = []
        ValueZ.append(z)
    
    NilaiBantu = 0
    for key, value in dataLatih.iterrows():
        for baris in range (len(ValueWjk)):
            for kolom in range (len(ValueWjk[0])):
                NilaiYin = ValueZ[baris][kolom] * ValueWjk[baris][kolom]
                NilaiBantu += NilaiYin
            NilaiYin = NilaiBantu + ValueW0[baris]
            Bantu.append(NilaiYin)
            y = [fungsi_aktivasi(i) for i in Bantu]
        ValueYin.append(Bantu)
        Bantu = []
        ValueY.append(y)
    print(ValueY)
    
    

In [100]:
y = Pengujian(dfNormalisasi, NilaiVij, NilaiV0, NilaiW0, NilaiWjk, ValueZin, ValueZ)

[[0.3519865716476145, 0.3514746194595914, 0.30080345739038644], [0.35182715780509777, 0.3513153117768821, 0.3006564691842559], [0.35166777697980217, 0.35115603720337285, 0.30050952193388464], [0.3515084292003928, 0.3509967957676388, 0.30036261565803907], [0.3513491144955071, 0.3508375874982271, 0.3002157503754554], [0.3511898328937541, 0.3506784124236567, 0.300068926104839], [0.351030584423715, 0.35051927057241844, 0.2999221428648651], [0.35087136911394273, 0.35036016197297515, 0.29977540067417835], [0.3507121869929623, 0.35020108665376143, 0.2996286995513928], [0.3505530380892705, 0.35004204464318384, 0.2994820395150922], [0.35039392243133627, 0.34988303596962045, 0.2993354205838298], [0.35023484004759997, 0.34972406066142175, 0.2991888427761281], [0.35007579096647423, 0.3495651187469094, 0.2990423061104796], [0.34991677521634323, 0.3494062102543773, 0.2988958106053459], [0.34975779282556313, 0.34924733521209084, 0.29874935627915844], [0.3495988438224618, 0.3490884936482873, 0.2986029

In [101]:
def one_hot_encoding(y):
    if(y.index(max(y)) == 0):
        target = 1
    elif(y.index(max(y)) == 1):
        target = 2
    elif(y.index(max(y)) == 2):
        target = 3

    return target

In [102]:
y_class = []
for x in ValueY:
    y = one_hot_encoding(x)
    y_class.append(y)
    
y_class

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [103]:
accu = 0
for i in range(0, len(y_class)):
    if(y_class[i] == ValueTarget[i]):
        accu += 1

acc = accu/len(y_class) * 100

acc

30.0